# How to Submit an entry to the Competition

### This notebook will show how to
- instantiate dataloader for the demo data
- instantiate pytorch model
- instantiate a trainer function
- load the model weights
- generate the submission file with the trained model

# Step 1) Instantiate the pre-trained model

### Imports

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

### Instantiate DataLoader

In [2]:
import os


filenames = ['./data/static26872-17-20-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

In [3]:
dataloaders

OrderedDict([('train',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7ffabb2840a0>)])),
             ('validation',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7ffabb284d60>)])),
             ('test',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7ffabb284cd0>)])),
             ('final_test',
              OrderedDict([('26872-17-20',
                            <torch.utils.data.dataloader.DataLoader at 0x7ffabb284d00>)]))])

# Instantiate State of the Art Model (SOTA)

In [7]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 64,
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

# load checkpoints

In [9]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_sota_model.pth"));

In [10]:
# important: setting the model to evaluation mode
model.eval();

---

# Step 2) Generate the submission file

In [13]:
# specify the dataset_name for the dataloader, in this case, it's the sensorium dataset
dataset_name = '26872-17-20'

# alternatively:
# dataset_name = list(dataloaders["train"].keys())[0]

In [14]:
# import the submission API

from sensorium.utility import submission

In [ ]:
# generate the submission file
submission.generate_submission_file(trained_model=model, 
                                    dataloaders=dataloaders,
                                    data_key=dataset_name,
                                    path="./submission_files/",
                                    device="cuda")

#### That's it.

In [ ]:
# now you can upload the "live_test"  and "final_test" submission files to the competition homepage. All the metrics will be automatically calculated.

---

In [8]:
import pandas as pd
pd.read_csv('./submission_files/example_submission_file.csv')

,trial_indices,image_ids,prediction,neuron_ids
0,126,2214,"[0.39909374713897705, 0.5152013301849365, 0.51...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
1,297,2214,"[0.3848317861557007, 0.5232589840888977, 0.504...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
2,597,2214,"[0.38958680629730225, 0.527167022228241, 0.512...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
3,852,2214,"[0.38263487815856934, 0.5397306084632874, 0.50...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
4,908,2214,"[0.38469237089157104, 0.5387403964996338, 0.50...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
...,...,...,...,...
994,2752,3487,"[0.0845494270324707, 0.14774799346923828, 0.27...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
995,3039,3487,"[0.08712136745452881, 0.1528913974761963, 0.25...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
996,4312,3487,"[0.08987849950790405, 0.16628813743591309, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."
997,4380,3487,"[0.08792203664779663, 0.16572076082229614, 0.2...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15..."


# The submission process

- now that the .csv file is generated, the participant can make an entry to the competition
- all that is left to do is:
 - register a new user on http://sensorium2022.net/ with a valid email address
 - upload the two .csv files
 - the competition score will be calculated automatically and will show up in the live leaderboard within a few seconds

- **Note**: Before June 20, our website does not accept submissions.
The competition does start officially on June 20. On this date, we're releasing the competition dataset, and submissions will be open.

---

---

# How we calculate the competition scores behind the scenes

- we will withhold the ground truth neuronal responses to the test set images in the actual competition
- here we show
 - how we extract the ground truth responses from the demo dataset (where the test set responses are present)
 - how the metrics of the competition are calculated

In [13]:
from sensorium import evaluate

In [14]:
# we load the dataset which contains the held-out "test" responses, and save them in the .csv format
# for the demo dataset that we provide here, these "test" responses are present

filename = ['./data/lurz2020/static20457-5-9-preproc0']
submission.generate_ground_truth_file(filename=filename,
                                      path='./ground_truth_files/ground_truth_file.csv')

### SOTA model results

In [15]:
ground_truth_file = './ground_truth_files/ground_truth_file.csv'
submission_file = './submission_files/sota_submission_file.csv'

In [16]:
out = evaluate(submission_file, ground_truth_file)

In [17]:
print("Results for the SOTA model:")
for metric, value in out.items():
    print(f"{metric}: {np.round(value, 3)}")

Results for the SOTA model:
Correlation (single trial): 0.302
Correlation (mean): 0.549
FEVE: 0.297
